## Question 1. dlt Version

In this homework, we will load the data from our FAQ to Qdrant

Let's install dlt with Qdrant support and Qdrant client:

```bash
pip install -q "dlt[qdrant]" "qdrant-client[fastembed]"
```

What's the version of dlt that you installed?



In [4]:
# !pip install -q "dlt[qdrant]" "qdrant-client[fastembed]"


In [3]:
import dlt
print(dlt.__version__)


1.13.0


## A1.

The version of dlt is 1.13.0

## Question 2. dlt pipeline


In [5]:
# Step 1: Annotate zoomcamp_data() with @dlt.resource

import dlt
import requests

@dlt.resource
def zoomcamp_data():
    docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
    docs_response = requests.get(docs_url)
    documents_raw = docs_response.json()

    for course in documents_raw:
        course_name = course['course']

        for doc in course['documents']:
            doc['course'] = course_name
            yield doc


In [6]:
# Step 2: Create pipeline and load data
from dlt.destinations import qdrant

qdrant_destination = qdrant(qd_path="db.qdrant")

pipeline = dlt.pipeline(
    pipeline_name="zoomcamp_pipeline",
    destination=qdrant_destination,
    dataset_name="zoomcamp_tagged_data"
)

load_info = pipeline.run(zoomcamp_data())
print(pipeline.last_trace)


Run started at 2025-07-12 09:46:30.829827+00:00 and COMPLETED in 10.90 seconds with 4 steps.
Step extract COMPLETED in 0.48 seconds.

Load package 1752313595.0191445 is EXTRACTED and NOT YET LOADED to the destination and contains no failed jobs

Step normalize COMPLETED in 0.09 seconds.
Normalized data for the following tables:
- zoomcamp_data: 948 row(s)
- _dlt_pipeline_state: 1 row(s)

Load package 1752313595.0191445 is NORMALIZED and NOT YET LOADED to the destination and contains no failed jobs

Step load COMPLETED in 6.14 seconds.
Pipeline zoomcamp_pipeline load step completed in 6.12 seconds
1 load package(s) were loaded to destination qdrant and into dataset zoomcamp_tagged_data
The qdrant destination used /workspaces/LLM-Zoomcamp/workshops/db.qdrant location to store data
Load package 1752313595.0191445 is LOADED and contains no failed jobs

Step run COMPLETED in 10.90 seconds.
Pipeline zoomcamp_pipeline load step completed in 6.12 seconds
1 load package(s) were loaded to destin

## A2.

948 rows inserted into the `zoomcamp_data` collection

## Question 3. Embeddings

When inserting the data, an embedding model was used. Which one?

You can find this out by inspecting the `meta.json` file created
in the target folder. During the data insertion process, a folder named db.qdrant will be created, and the meta.json file will be located inside this folder.



In [9]:
import json

with open("db.qdrant/meta.json") as f:
    meta = json.load(f)

print(json.dumps(meta, indent=2))

{
  "collections": {
    "zoomcamp_tagged_data": {
      "vectors": {
        "fast-bge-small-en": {
          "size": 384,
          "distance": "Cosine",
          "hnsw_config": null,
          "quantization_config": null,
          "on_disk": null,
          "datatype": null,
          "multivector_config": null
        }
      },
      "shard_number": null,
      "sharding_method": null,
      "replication_factor": null,
      "write_consistency_factor": null,
      "on_disk_payload": null,
      "hnsw_config": null,
      "wal_config": null,
      "optimizers_config": null,
      "init_from": null,
      "quantization_config": null,
      "sparse_vectors": null,
      "strict_mode_config": null
    },
    "zoomcamp_tagged_data__dlt_loads": {
      "vectors": {
        "fast-bge-small-en": {
          "size": 384,
          "distance": "Cosine",
          "hnsw_config": null,
          "quantization_config": null,
          "on_disk": null,
          "datatype": null,
          "m

In [ ]:
"""{
  "vectors": {
    "fast-bge-small-en": {
      "size": 384,
      "distance": "Cosine"
    }
  }
}
"""

## A3.

Model used is "fast-bge-small-en"